# 第52章: レンズ歪みと補正

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] レンズ歪みの物理的原因を理解する
- [ ] 放射歪み・接線歪みの数学モデルを実装できる
- [ ] Brown-Conradyモデルを説明できる
- [ ] 歪み補正アルゴリズムを実装できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ Notebook 50（光学基礎）
- ✅ Notebook 51（ピンホールカメラモデル）

⏱️ **推定学習時間**: 90-120分  
📊 **難易度**: ★★★☆☆（中級）  
🎓 **カテゴリ**: 理論 + 実践

---

## 🌟 はじめに

前章で学んだピンホールカメラモデルは「理想的なカメラ」を仮定しています。しかし、実際のカメラレンズには必ず**歪み（Distortion）**が存在します。

```
理想的なピンホール              実際のレンズ
┌─────────────┐              ┌─────────────┐
│ ┌─────────┐ │              │ ╭─────────╮ │
│ │         │ │              │ │         │ │
│ │    ■    │ │      →      │ ╰─────────╯ │  ← 歪み!
│ │         │ │              │             │
│ └─────────┘ │              │             │
└─────────────┘              └─────────────┘
   直線が直線                  直線が曲がる
```

この章では：
- なぜ歪みが生じるのか
- 歪みをどのようにモデル化するか
- どのように補正するか

を学びます。

In [ ]:
# ============================================================
# 環境設定
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
import warnings

warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. レンズ歪みの原因

### 1.1 収差（Aberration）の種類

実際のレンズは様々な**収差**を持ちます：

| 収差 | 説明 | 影響 |
|------|------|------|
| **球面収差** | レンズの端と中心で焦点が異なる | ボケ、シャープネス低下 |
| **コマ収差** | 点が彗星状に伸びる | 周辺部のボケ |
| **非点収差** | 縦と横で焦点が異なる | 線が曲がる |
| **像面湾曲** | 像面が平面にならない | 周辺部のボケ |
| **歪曲収差** | 直線が曲がる | **幾何学的歪み** ←主な対象 |
| **色収差** | 波長により焦点が異なる | 色のにじみ |

Computer Visionで最も問題になるのは**歪曲収差（Distortion）**です。

In [ ]:
# ============================================================
# 収差の種類の可視化
# ============================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# 理想的な像
ax = axes[0]
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(np.cos(theta), np.sin(theta), 'b-', linewidth=2)
ax.plot(0, 0, 'b.', markersize=15)
ax.set_title('理想的な像（点）', fontsize=12, fontweight='bold')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.axis('off')

# 球面収差
ax = axes[1]
for i in range(5):
    r = 0.1 + i * 0.15
    ax.plot(r * np.cos(theta), r * np.sin(theta), 'b-', alpha=0.5 - i*0.08)
ax.set_title('球面収差\n（点がボケる）', fontsize=12, fontweight='bold')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.axis('off')

# コマ収差
ax = axes[2]
t = np.linspace(0, 1, 50)
x = t * 1.5
y = 0.3 * np.sin(np.pi * t) * (1 - t)
ax.fill(np.concatenate([x, x[::-1]]), np.concatenate([y, -y[::-1]]), 'b', alpha=0.5)
ax.plot(0, 0, 'b.', markersize=15)
ax.set_title('コマ収差\n（彗星状に伸びる）', fontsize=12, fontweight='bold')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.axis('off')

# 非点収差
ax = axes[3]
ax.plot([-0.8, 0.8], [0, 0], 'b-', linewidth=3, alpha=0.7, label='水平')
ax.plot([0, 0], [-0.5, 0.5], 'r-', linewidth=3, alpha=0.7, label='垂直')
ax.set_title('非点収差\n（縦横で焦点が異なる）', fontsize=12, fontweight='bold')
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.axis('off')

# 歪曲収差（樽型）
ax = axes[4]
# グリッド
for i in np.linspace(-1, 1, 5):
    x = np.linspace(-1, 1, 50)
    y = np.full_like(x, i)
    r = np.sqrt(x**2 + y**2)
    k = -0.2  # 樽型歪み
    factor = 1 + k * r**2
    ax.plot(x * factor, y * factor, 'b-', linewidth=1)
    ax.plot(y * factor, x * factor, 'b-', linewidth=1)
ax.set_title('歪曲収差（樽型）\n（直線が曲がる）', fontsize=12, fontweight='bold')
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.axis('off')

# 色収差
ax = axes[5]
ax.plot(0.05 * np.cos(theta), 0.05 * np.sin(theta), 'r-', linewidth=3, alpha=0.7)
ax.plot(0.03 * np.cos(theta), 0.03 * np.sin(theta), 'g-', linewidth=3, alpha=0.7)
ax.plot(0.01 * np.cos(theta), 0.01 * np.sin(theta), 'b-', linewidth=3, alpha=0.7)
ax.set_title('色収差\n（色によって焦点が異なる）', fontsize=12, fontweight='bold')
ax.set_xlim(-0.1, 0.1)
ax.set_ylim(-0.1, 0.1)
ax.set_aspect('equal')
ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 Computer Visionでは主に歪曲収差（Distortion）を補正します")

---

## 2. 放射歪み（Radial Distortion）

### 2.1 放射歪みの種類

放射歪みは画像中心からの距離に依存する歪みです：

| 種類 | 特徴 | 原因 |
|------|------|------|
| **樽型歪み（Barrel）** | 直線が外側に膨らむ | 広角レンズで顕著 |
| **糸巻き型歪み（Pincushion）** | 直線が内側に縮む | 望遠レンズで顕著 |
| **口髭型歪み（Mustache）** | 中心付近で樽型、周辺で糸巻き型 | 複合レンズ |

```
  理想        樽型        糸巻き型      口髭型
┌─────┐    ╭─────╮    ╱─────╲    ╭─────╮
│     │    │     │    │     │    ╯     ╰
│     │    │     │    │     │    │     │
│     │    │     │    │     │    ╮     ╭
└─────┘    ╰─────╯    ╲─────╱    ╰─────╯
```

In [ ]:
# ============================================================
# 放射歪みの3種類を可視化
# ============================================================

def apply_radial_distortion(x, y, k1, k2=0, k3=0):
    """
    放射歪みを適用
    
    x_d = x * (1 + k1*r^2 + k2*r^4 + k3*r^6)
    y_d = y * (1 + k1*r^2 + k2*r^4 + k3*r^6)
    """
    r2 = x**2 + y**2
    r4 = r2**2
    r6 = r2**3
    
    factor = 1 + k1 * r2 + k2 * r4 + k3 * r6
    
    x_d = x * factor
    y_d = y * factor
    
    return x_d, y_d


def create_grid(n=10, extent=1.0):
    """グリッドを生成"""
    lines = []
    # 水平線
    for y_val in np.linspace(-extent, extent, n):
        x = np.linspace(-extent, extent, 100)
        y = np.full_like(x, y_val)
        lines.append((x, y))
    # 垂直線
    for x_val in np.linspace(-extent, extent, n):
        y = np.linspace(-extent, extent, 100)
        x = np.full_like(y, x_val)
        lines.append((x, y))
    return lines


fig, axes = plt.subplots(1, 4, figsize=(18, 5))

distortion_types = [
    ('理想（歪みなし）', 0, 0, 0),
    ('樽型歪み (k1 < 0)', -0.3, 0, 0),
    ('糸巻き型歪み (k1 > 0)', 0.3, 0, 0),
    ('口髭型歪み', -0.5, 0.3, 0),
]

grid = create_grid(n=9, extent=1.0)

for ax, (title, k1, k2, k3) in zip(axes, distortion_types):
    for x, y in grid:
        x_d, y_d = apply_radial_distortion(x, y, k1, k2, k3)
        ax.plot(x_d, y_d, 'b-', linewidth=1)
    
    # 中心点
    ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
    
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

### 2.2 放射歪みの数学モデル

正規化画像座標 $(x, y)$ から歪んだ座標 $(x_d, y_d)$ への変換：

$$x_d = x \cdot (1 + k_1 r^2 + k_2 r^4 + k_3 r^6)$$
$$y_d = y \cdot (1 + k_1 r^2 + k_2 r^4 + k_3 r^6)$$

ここで：
- $r^2 = x^2 + y^2$（中心からの距離の2乗）
- $k_1, k_2, k_3$：放射歪み係数

**歪み係数の意味:**
- $k_1 < 0$：樽型歪み
- $k_1 > 0$：糸巻き型歪み
- 高次項（$k_2, k_3$）：より細かい補正

In [ ]:
# ============================================================
# 放射歪みの強度と距離の関係
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 歪みファクターのプロット
ax = axes[0]

r = np.linspace(0, 1.5, 100)
r2 = r**2
r4 = r**4
r6 = r**6

distortion_configs = [
    ('理想 (k=0)', 0, 0, 0, 'black'),
    ('軽度の樽型', -0.1, 0, 0, 'blue'),
    ('強度の樽型', -0.3, 0, 0, 'red'),
    ('軽度の糸巻き', 0.1, 0, 0, 'green'),
    ('口髭型', -0.5, 0.3, 0, 'purple'),
]

for label, k1, k2, k3, color in distortion_configs:
    factor = 1 + k1 * r2 + k2 * r4 + k3 * r6
    ax.plot(r, factor, label=label, color=color, linewidth=2)

ax.axhline(1, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('中心からの距離 r', fontsize=12)
ax.set_ylabel('歪みファクター', fontsize=12)
ax.set_title('放射歪みファクター vs 距離', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1.5)

# 右: 実際の変位量
ax = axes[1]

for label, k1, k2, k3, color in distortion_configs:
    factor = 1 + k1 * r2 + k2 * r4 + k3 * r6
    displacement = r * (factor - 1)  # 変位量
    ax.plot(r, displacement, label=label, color=color, linewidth=2)

ax.axhline(0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('中心からの距離 r', fontsize=12)
ax.set_ylabel('変位量 (r × (factor - 1))', fontsize=12)
ax.set_title('放射方向の変位量', fontsize=12, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 1.5)

plt.tight_layout()
plt.show()

print("💡 歪みは中心から離れるほど大きくなります（r^2に比例）")

---

## 3. 接線歪み（Tangential Distortion）

### 3.1 接線歪みとは

接線歪みは、レンズとセンサーが完全に平行でない場合に生じます：

```
   理想的な配置              傾いた配置
   ┌─────────┐              ┌─────────┐
   │  レンズ  │              │  レンズ  │
   └─────────┘              └────╲────┘
        │                        ╲
        │                         ╲
   ┌─────────┐                ┌────╲────┐
   │ センサー │                │ センサー │
   └─────────┘                └─────────┘
   光軸が垂直                  光軸が傾いている
```

### 3.2 接線歪みの数学モデル

$$x_d = x + [2p_1 xy + p_2(r^2 + 2x^2)]$$
$$y_d = y + [p_1(r^2 + 2y^2) + 2p_2 xy]$$

- $p_1, p_2$：接線歪み係数

In [ ]:
# ============================================================
# 接線歪みの可視化
# ============================================================

def apply_tangential_distortion(x, y, p1, p2):
    """
    接線歪みを適用
    
    x_d = x + [2*p1*x*y + p2*(r^2 + 2*x^2)]
    y_d = y + [p1*(r^2 + 2*y^2) + 2*p2*x*y]
    """
    r2 = x**2 + y**2
    
    x_d = x + 2 * p1 * x * y + p2 * (r2 + 2 * x**2)
    y_d = y + p1 * (r2 + 2 * y**2) + 2 * p2 * x * y
    
    return x_d, y_d


fig, axes = plt.subplots(1, 4, figsize=(18, 5))

tangential_configs = [
    ('理想（歪みなし）', 0, 0),
    ('p1 > 0（垂直方向のズレ）', 0.1, 0),
    ('p2 > 0（水平方向のズレ）', 0, 0.1),
    ('p1, p2 両方', 0.05, 0.05),
]

grid = create_grid(n=9, extent=1.0)

for ax, (title, p1, p2) in zip(axes, tangential_configs):
    for x, y in grid:
        x_d, y_d = apply_tangential_distortion(x, y, p1, p2)
        ax.plot(x_d, y_d, 'b-', linewidth=1)
    
    ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 接線歪みはレンズの取り付け精度に依存し、通常は放射歪みより小さい")

---

## 4. Brown-Conradyモデル

### 4.1 完全な歪みモデル

**Brown-Conrady**モデルは、放射歪みと接線歪みを組み合わせた標準的なモデルです：

$$x_d = x(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + 2p_1 xy + p_2(r^2 + 2x^2)$$
$$y_d = y(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + p_1(r^2 + 2y^2) + 2p_2 xy$$

### 4.2 OpenCVでの表現

OpenCVでは歪み係数を以下の形式で表現します：

```
dist_coeffs = [k1, k2, p1, p2, k3, k4, k5, k6, ...]
```

通常は `[k1, k2, p1, p2, k3]` の5つで十分です。

In [ ]:
# ============================================================
# Brown-Conradyモデルの完全な実装
# ============================================================

def apply_distortion(x, y, k1=0, k2=0, k3=0, p1=0, p2=0):
    """
    Brown-Conradyモデルによる歪みを適用
    
    Parameters:
    -----------
    x, y : array
        正規化画像座標
    k1, k2, k3 : float
        放射歪み係数
    p1, p2 : float
        接線歪み係数
    
    Returns:
    --------
    x_d, y_d : array
        歪んだ座標
    """
    r2 = x**2 + y**2
    r4 = r2**2
    r6 = r2**3
    
    # 放射歪み
    radial = 1 + k1 * r2 + k2 * r4 + k3 * r6
    
    # 接線歪み
    dx_tangential = 2 * p1 * x * y + p2 * (r2 + 2 * x**2)
    dy_tangential = p1 * (r2 + 2 * y**2) + 2 * p2 * x * y
    
    # 合成
    x_d = x * radial + dx_tangential
    y_d = y * radial + dy_tangential
    
    return x_d, y_d


# 典型的なカメラの歪み係数（広角レンズの例）
typical_wide_angle = {
    'k1': -0.28,
    'k2': 0.07,
    'k3': 0.0,
    'p1': 0.001,
    'p2': -0.002
}

# GoPro風の魚眼に近いレンズ
typical_fisheye = {
    'k1': -0.4,
    'k2': 0.15,
    'k3': -0.02,
    'p1': 0.0,
    'p2': 0.0
}

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

configs = [
    ('理想（歪みなし）', {}),
    ('広角レンズ（典型的）', typical_wide_angle),
    ('アクションカメラ風', typical_fisheye),
]

grid = create_grid(n=11, extent=1.0)

for ax, (title, params) in zip(axes, configs):
    for x, y in grid:
        x_d, y_d = apply_distortion(x, y, **params)
        ax.plot(x_d, y_d, 'b-', linewidth=1)
    
    ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    # 係数を表示
    if params:
        param_str = ', '.join([f'{k}={v}' for k, v in params.items() if v != 0])
        ax.text(0, -1.35, param_str, fontsize=9, ha='center', 
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    ax.axis('off')

plt.tight_layout()
plt.show()

---

## 5. 歪み補正アルゴリズム

### 5.1 前方マッピング vs 逆マッピング

**前方マッピング（Forward Mapping）:**
- 歪んでいない座標 → 歪んだ座標
- 問題：出力画像に「穴」ができる可能性

**逆マッピング（Inverse Mapping）:**
- 歪んだ座標 → 歪んでいない座標
- 出力画像の各ピクセルに対応する入力座標を計算
- 実用的な補正で使用

```
前方マッピング:              逆マッピング:
入力(歪みなし) → 出力(歪み)    出力(歪みなし) ← 入力(歪み)

問題: 穴ができる              解決: 各ピクセルを埋められる
  ┌───┐                       ┌───┐
  │○○○│ → ┌───┐              │●●●│ ← ┌───┐
  │○○○│   │● ●│              │●●●│   │○○○│
  │○○○│   │ ● │              │●●●│   │○○○│
  └───┘   └───┘              └───┘   └───┘
```

In [ ]:
# ============================================================
# 歪み補正（逆マッピング）の実装
# ============================================================

def undistort_point_iterative(x_d, y_d, k1, k2=0, k3=0, p1=0, p2=0, iterations=10):
    """
    歪んだ座標から歪みのない座標を反復法で計算（逆マッピング）
    
    歪みの式: x_d = f(x) を x について解く
    反復: x_{n+1} = x_d - (f(x_n) - x_n) ≈ x_d / factor
    """
    # 初期値は歪んだ座標そのまま
    x, y = x_d.copy(), y_d.copy()
    
    for _ in range(iterations):
        r2 = x**2 + y**2
        r4 = r2**2
        r6 = r2**3
        
        # 放射歪みファクター
        radial = 1 + k1 * r2 + k2 * r4 + k3 * r6
        
        # 接線歪み成分
        dx_tan = 2 * p1 * x * y + p2 * (r2 + 2 * x**2)
        dy_tan = p1 * (r2 + 2 * y**2) + 2 * p2 * x * y
        
        # 更新
        x = (x_d - dx_tan) / radial
        y = (y_d - dy_tan) / radial
    
    return x, y


# 補正のデモ
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 歪みパラメータ
k1, k2 = -0.3, 0.05

grid = create_grid(n=11, extent=1.0)

# 左: 元のグリッド（理想）
ax = axes[0]
for x, y in grid:
    ax.plot(x, y, 'b-', linewidth=1)
ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_title('1. 理想的なグリッド', fontsize=12, fontweight='bold')
ax.axis('off')

# 中央: 歪んだグリッド
ax = axes[1]
for x, y in grid:
    x_d, y_d = apply_distortion(x, y, k1=k1, k2=k2)
    ax.plot(x_d, y_d, 'r-', linewidth=1)
ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_title(f'2. 歪んだグリッド\n(k1={k1}, k2={k2})', fontsize=12, fontweight='bold')
ax.axis('off')

# 右: 補正後
ax = axes[2]
for x, y in grid:
    # まず歪ませる
    x_d, y_d = apply_distortion(x, y, k1=k1, k2=k2)
    # 補正する
    x_u, y_u = undistort_point_iterative(x_d, y_d, k1=k1, k2=k2)
    ax.plot(x_u, y_u, 'g-', linewidth=1)
ax.plot(0, 0, 'r+', markersize=15, markeredgewidth=2)
ax.set_xlim(-1.5, 1.5)
ax.set_ylim(-1.5, 1.5)
ax.set_aspect('equal')
ax.set_title('3. 補正後のグリッド', fontsize=12, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 反復法により、歪んだ座標から元の座標を復元できます")

### 5.2 画像全体の歪み補正

In [ ]:
# ============================================================
# 画像全体の歪み補正シミュレーション
# ============================================================

def create_test_image(width=400, height=300):
    """チェッカーボードパターンのテスト画像を生成"""
    image = np.zeros((height, width))
    
    # チェッカーボード
    block_size = 40
    for i in range(0, height, block_size):
        for j in range(0, width, block_size):
            if ((i // block_size) + (j // block_size)) % 2 == 0:
                image[i:i+block_size, j:j+block_size] = 1
    
    return image


def apply_distortion_to_image(image, k1, k2=0, p1=0, p2=0):
    """画像に歪みを適用（前方マッピング、簡易版）"""
    h, w = image.shape[:2]
    
    # 出力画像
    output = np.zeros_like(image)
    
    # 中心を原点とした正規化座標に変換
    cx, cy = w / 2, h / 2
    
    for v in range(h):
        for u in range(w):
            # 正規化座標
            x = (u - cx) / max(w, h)
            y = (v - cy) / max(w, h)
            
            # 歪みを適用
            x_d, y_d = apply_distortion(x, y, k1=k1, k2=k2, p1=p1, p2=p2)
            
            # ピクセル座標に戻す
            u_d = int(x_d * max(w, h) + cx)
            v_d = int(y_d * max(w, h) + cy)
            
            if 0 <= u_d < w and 0 <= v_d < h:
                output[v_d, u_d] = image[v, u]
    
    return output


def undistort_image(distorted_image, k1, k2=0, p1=0, p2=0):
    """歪んだ画像を補正（逆マッピング）"""
    h, w = distorted_image.shape[:2]
    
    # 出力画像
    output = np.zeros_like(distorted_image)
    
    cx, cy = w / 2, h / 2
    
    for v in range(h):
        for u in range(w):
            # 出力画像の座標を正規化
            x = (u - cx) / max(w, h)
            y = (v - cy) / max(w, h)
            
            # この座標に対応する歪み画像上の座標を計算（歪みを適用）
            x_d, y_d = apply_distortion(x, y, k1=k1, k2=k2, p1=p1, p2=p2)
            
            # ピクセル座標に戻す
            u_d = int(x_d * max(w, h) + cx)
            v_d = int(y_d * max(w, h) + cy)
            
            if 0 <= u_d < w and 0 <= v_d < h:
                output[v, u] = distorted_image[v_d, u_d]
    
    return output


# テスト
test_image = create_test_image(400, 300)
k1, k2 = -0.4, 0.1

# 歪み画像を作成
distorted = apply_distortion_to_image(test_image, k1, k2)

# 補正
corrected = undistort_image(distorted, k1, k2)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].imshow(test_image, cmap='gray')
axes[0].set_title('元画像（理想）', fontsize=12, fontweight='bold')
axes[0].axis('off')

axes[1].imshow(distorted, cmap='gray')
axes[1].set_title(f'歪み画像 (k1={k1}, k2={k2})', fontsize=12, fontweight='bold')
axes[1].axis('off')

axes[2].imshow(corrected, cmap='gray')
axes[2].set_title('補正後', fontsize=12, fontweight='bold')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("💡 実際のアプリケーションではcv2.undistort()を使用します")

---

## 6. 魚眼レンズモデル

広角180度以上の**魚眼レンズ**は、Brown-Conradyモデルでは正確にモデル化できません。

### 6.1 魚眼射影モデルの種類

| モデル | 射影式 | 特徴 |
|--------|--------|------|
| 等距離（Equidistant） | $r = f \cdot \theta$ | 角度が距離に比例 |
| 等立体角（Equisolid） | $r = 2f \sin(\theta/2)$ | 立体角が保存 |
| 正射影（Orthographic） | $r = f \sin(\theta)$ | 球面の正射影 |
| 立体射影（Stereographic） | $r = 2f \tan(\theta/2)$ | 円が円に写る |

In [ ]:
# ============================================================
# 魚眼レンズの射影モデル比較
# ============================================================

def perspective_projection(theta):
    """通常の透視射影: r = f * tan(theta)"""
    return np.tan(theta)

def equidistant_projection(theta):
    """等距離射影: r = f * theta"""
    return theta

def equisolid_projection(theta):
    """等立体角射影: r = 2f * sin(theta/2)"""
    return 2 * np.sin(theta / 2)

def orthographic_projection(theta):
    """正射影: r = f * sin(theta)"""
    return np.sin(theta)

def stereographic_projection(theta):
    """立体射影: r = 2f * tan(theta/2)"""
    return 2 * np.tan(theta / 2)


fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: 射影関数の比較
ax = axes[0]

theta = np.linspace(0, np.pi/2 - 0.1, 100)  # 0° to 90°
theta_deg = np.degrees(theta)

projections = [
    ('透視射影', perspective_projection, 'black'),
    ('等距離', equidistant_projection, 'blue'),
    ('等立体角', equisolid_projection, 'green'),
    ('正射影', orthographic_projection, 'red'),
    ('立体射影', stereographic_projection, 'purple'),
]

for name, func, color in projections:
    r = func(theta)
    ax.plot(theta_deg, r, label=name, color=color, linewidth=2)

ax.set_xlabel('入射角 θ (度)', fontsize=12)
ax.set_ylabel('像の半径 r / f', fontsize=12)
ax.set_title('魚眼射影モデルの比較', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 90)
ax.set_ylim(0, 3)

# 右: 画像上での見え方
ax = axes[1]

# 等角度間隔の同心円を描画
theta_circles = np.linspace(0, np.pi/2, 10)
phi = np.linspace(0, 2*np.pi, 100)

for name, func, color in projections[1:]:  # 透視射影を除く
    for t in theta_circles[1:]:
        r = func(t)
        x = r * np.cos(phi)
        y = r * np.sin(phi)
        ax.plot(x, y, color=color, linewidth=0.5, alpha=0.5)
    
    # 最外周にラベル
    r_max = func(theta_circles[-1])
    ax.annotate(name, xy=(r_max * 0.7, r_max * 0.7), fontsize=9, color=color)

ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.set_title('等角度間隔の同心円\n(各射影モデルでの見え方)', fontsize=12, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 魚眼レンズは通常のBrown-Conradyモデルでは不十分、専用モデルが必要")

---

## 7. 実践：歪み係数の影響を理解する

In [ ]:
# ============================================================
# インタラクティブな歪みパラメータ探索
# ============================================================

fig, axes = plt.subplots(2, 4, figsize=(18, 10))

# k1の影響
k1_values = [-0.4, -0.2, 0.2, 0.4]
for ax, k1 in zip(axes[0], k1_values):
    for x, y in grid:
        x_d, y_d = apply_distortion(x, y, k1=k1)
        ax.plot(x_d, y_d, 'b-', linewidth=1)
    ax.plot(0, 0, 'r+', markersize=10, markeredgewidth=2)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(f'k1 = {k1}', fontsize=11, fontweight='bold')
    ax.axis('off')

axes[0, 0].set_ylabel('k1 (放射歪み)', fontsize=12)

# k2の影響（k1=-0.2固定）
k2_values = [-0.1, 0, 0.1, 0.2]
for ax, k2 in zip(axes[1], k2_values):
    for x, y in grid:
        x_d, y_d = apply_distortion(x, y, k1=-0.2, k2=k2)
        ax.plot(x_d, y_d, 'g-', linewidth=1)
    ax.plot(0, 0, 'r+', markersize=10, markeredgewidth=2)
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.set_title(f'k1=-0.2, k2={k2}', fontsize=11, fontweight='bold')
    ax.axis('off')

axes[1, 0].set_ylabel('k2 (高次放射歪み)', fontsize=12)

plt.tight_layout()
plt.show()

print("💡 k1は低次の歪み、k2は高次項で細かい補正を行います")

---

## まとめ

### 🎯 このノートブックで学んだこと

**レンズ歪みの種類**
- ✓ 放射歪み: 樽型（k1 < 0）、糸巻き型（k1 > 0）
- ✓ 接線歪み: レンズの傾きによる

**Brown-Conradyモデル**
- ✓ $x_d = x(1 + k_1 r^2 + k_2 r^4 + k_3 r^6) + 2p_1 xy + p_2(r^2 + 2x^2)$
- ✓ OpenCV表記: `[k1, k2, p1, p2, k3]`

**歪み補正**
- ✓ 逆マッピングによる補正
- ✓ 反復法による座標変換

### 📊 歪みパラメータまとめ

| パラメータ | 正の値 | 負の値 |
|------------|--------|--------|
| k1 | 糸巻き型歪み | 樽型歪み |
| k2, k3 | 高次補正項 | 高次補正項 |
| p1, p2 | 接線歪み | 接線歪み |

---

## 🎓 自己評価クイズ

### Q1: k1 = -0.3 の場合、どのような歪みが生じますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 樽型歪み（Barrel Distortion）

k1 < 0 の場合、画像の周辺部が中心に向かって押し込まれ、直線が外側に膨らんで見えます。広角レンズで顕著です。

</details>

---

### Q2: 接線歪みの原因は何ですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: レンズとセンサー（撮像面）が完全に平行でないこと

製造・組み立て時の誤差により、光軸がセンサーに対して垂直でない場合に生じます。

</details>

---

### Q3: なぜ歪み補正には逆マッピングを使うのですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 出力画像のすべてのピクセルを埋めることができるから

前方マッピングでは、入力画像の各ピクセルを出力画像に写すため、出力画像に「穴」ができる可能性があります。逆マッピングでは、出力画像の各ピクセルに対応する入力座標を計算するため、確実にすべてのピクセルを埋められます。

</details>

---

### ✅ 学習チェックリスト

- [ ] 樽型歪みと糸巻き型歪みの違いを説明できる
- [ ] Brown-Conradyモデルの式を書ける
- [ ] 歪み係数 k1, k2, p1, p2 の意味を説明できる
- [ ] 前方マッピングと逆マッピングの違いを理解している

---

**次のステップ**: Notebook 53で、**3D座標変換と剛体運動**を学びます！

回転行列、クォータニオン、SE(3)群など、3D空間での変換を詳しく学びます。